In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import sys
sys.path.append('../')

In [4]:
import os
import json
import pandas as pd
import warnings
import random
import gc
from typing import Dict, List


In [5]:

import torch
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    set_seed,
    GenerationConfig
)
from peft import LoraConfig, get_peft_model
import evaluate

In [6]:
import os
import json
import pandas as pd
import warnings
import random
import gc
from typing import Dict, List

import torch
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    set_seed,
    GenerationConfig
)
from peft import LoraConfig, get_peft_model
import evaluate


In [7]:
from src.preprocess.deer import DeerToTriplets

In [8]:
from src.preprocess.utils import to_text

In [55]:
from src.models.inference.inference import build_inference_prompt, generate_answers_with, PromptAnswerCollator
from src.models.sft.lora import  check_model_parameters, finetune_model_with_lora

In [10]:
from src.models.evaluate import eval_metrics, print_compare

In [11]:
from src.utils.io.read import RawDataReader

In [12]:
from src.settings import Settings

In [13]:
import src.config as cfg 

In [14]:
rdr = RawDataReader(Settings.paths.RAW_DATA_PATH)

In [15]:
ir_triplets_dataset = rdr.read_ir_triplets()

In [16]:
deer_dataset = rdr.read_deer()

In [17]:
deer_to_triplets_converter = DeerToTriplets()

In [18]:
deer_to_triplets_converter.process(deer_dataset)

In [19]:
os.environ["HUGGINGFACE_HUB_TOKEN"] =  cfg.HUGGINGFACE_HUB_TOKEN

In [20]:
HF_TOKEN = os.environ["HUGGINGFACE_HUB_TOKEN"]

In [21]:
# from huggingface_hub import notebook_login
# notebook_login()

In [22]:


od_val_data = deer_to_triplets_converter.triplets

In [23]:
# Configuration - GPU Optimized
MODEL_ID =cfg.MODEL_ID  
OUTPUT_DIR =cfg.OUTPUT_DIR  
SEED =cfg.SEED  
VAL_FRACTION =cfg.VAL_FRACTION  
MAX_SEQ_LEN =cfg.MAX_SEQ_LEN  

# LoRA Configuration - Larger for GPU
LORA_R =cfg.LORA_R  
LORA_ALPHA =cfg.LORA_ALPHA  
LORA_DROPOUT =cfg.LORA_DROPOUT  
TARGET_MODULES =cfg.TARGET_MODULES  

# Training Configuration - GPU Optimized
LR =cfg.LR  
EPOCHS =cfg.EPOCHS  
BATCH_SIZE =cfg.BATCH_SIZE  
GRAD_ACCUM =cfg.GRAD_ACCUM  
LOG_STEPS =cfg.LOG_STEPS  
EVAL_STEPS =cfg.EVAL_STEPS  
SAVE_STEPS =cfg.SAVE_STEPS  
GEN_MAX_NEW_TOKENS =cfg.GEN_MAX_NEW_TOKENS  


In [24]:
data = ir_triplets_dataset 

In [25]:


set_seed(SEED)

# Split data
random.Random(SEED).shuffle(data)
split_idx = int(len(data) * (1 - VAL_FRACTION))
train_raw, id_val_raw = data[:split_idx], data[split_idx:]

od_val_raw = od_val_data
# train_raw = data #, _ = data[:split_idx], data[split_idx:]
# val_raw = val_data
 


print(f"Training examples: {len(train_raw)}")
print(f"In Distribution Validation examples: {len(id_val_raw)}")
print(f"Out of Distribution Validation examples: {len(od_val_raw)}")


Training examples: 1264
In Distribution Validation examples: 543
Out of Distribution Validation examples: 273


In [26]:

train_ds = Dataset.from_list([to_text(x) for x in train_raw])
id_val_ds = Dataset.from_list([to_text(x) for x in id_val_raw])
id_ds = DatasetDict({"train": train_ds, "validation": id_val_ds})

od_val_ds = Dataset.from_list([to_text(x) for x in od_val_raw])
od_ds = DatasetDict({"train": train_ds, "validation": od_val_ds})


In [27]:
id_val_prompts = id_ds["validation"]["prompt"]
od_val_prompts = od_ds["validation"]["prompt"]



In [28]:
def from_ds_to_triplets(ds):
    
    prompts = ds["validation"]["prompt"]
    obs = [p.split("Question:\n")[0].replace("Training Observations:\n", "").strip() for p in prompts]
    qs = [p.split("Question:\n")[1].split("\n\nAnswer:\n")[0].strip() for p in prompts]
    refs = [x.strip() for x in ds["validation"]["response"]]
    return prompts,  obs, qs, refs
    

In [29]:
id_val_prompts, id_val_obs, id_val_qs, id_val_refs = from_ds_to_triplets(id_ds )
od_val_prompts, od_val_obs, od_val_qs, od_val_refs = from_ds_to_triplets(od_ds )

In [30]:
# tokenizer = AutoTokenizer.from_pretrained(
#     MODEL_ID,
#     use_fast=True,
#     trust_remote_code=True,
#     token=False   # force anonymous; ignore any stored/env tokens
# )

In [31]:

# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True, token=HF_TOKEN)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Load evaluation metrics
rouge = evaluate.load("rouge")
bleu = evaluate.load("sacrebleu")



In [32]:
baseline_model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)
# (optional) also wrap .generate in autocast(bfloat16) as in A)


In [33]:
print("Generating baseline predictions...")
id_baseline_preds = generate_answers_with(baseline_model,tokenizer,id_val_obs, id_val_qs)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Generating baseline predictions...


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info

In [34]:
od_baseline_preds = generate_answers_with(baseline_model,tokenizer,od_val_obs, od_val_qs)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info

In [35]:

id_baseline_metrics = eval_metrics(id_baseline_preds, id_val_refs, rouge, bleu)
od_baseline_metrics = eval_metrics(od_baseline_preds, od_val_refs, rouge, bleu)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [36]:
print("Baseline metrics:", id_baseline_metrics)


Baseline metrics: {'rouge1': np.float64(0.02747859826509425), 'rouge2': np.float64(0.002061315914361044), 'rougeL': np.float64(0.022668664160191032), 'rougeLsum': np.float64(0.022727330004289076), 'bleu': 0.02064748741328389}


In [37]:
print("Baseline metrics:", od_baseline_metrics)


Baseline metrics: {'rouge1': np.float64(0.008275037125781669), 'rouge2': np.float64(0.0006343796259762647), 'rougeL': np.float64(0.005853888639777592), 'rougeLsum': np.float64(0.005881398679732773), 'bleu': 0.028309692089969404}


In [38]:



# Save baseline predictions
os.makedirs(OUTPUT_DIR, exist_ok=True)
with open(os.path.join(OUTPUT_DIR, "id_val_predictions_baseline.jsonl"), "w") as f:
    for obs, q, ref, pred in zip(id_val_obs, id_val_qs, id_val_refs, id_baseline_preds):
        f.write(json.dumps({
            "Training Observations": obs,
            "Question": q,
            "Reference": ref,
            "Prediction": pred
        }, ensure_ascii=False) + "\n")

# Free baseline model memory
# del baseline_model
gc.collect()
torch.cuda.empty_cache()  # Clear GPU memory


In [39]:
# Save baseline predictions
os.makedirs(OUTPUT_DIR, exist_ok=True)
with open(os.path.join(OUTPUT_DIR, "od_val_predictions_baseline.jsonl"), "w") as f:
    for obs, q, ref, pred in zip(od_val_obs, od_val_qs, od_val_refs, od_baseline_preds):
        f.write(json.dumps({
            "Training Observations": obs,
            "Question": q,
            "Reference": ref,
            "Prediction": pred
        }, ensure_ascii=False) + "\n")

# Free baseline model memory
# del baseline_model
gc.collect()
torch.cuda.empty_cache()  # Clear GPU memory

In [40]:
import torch
import torch, sys, platform
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [57]:
model = finetune_model_with_lora(
                    model_id=MODEL_ID,
                    train_dataset= train_ds,
                    tokenizer=tokenizer,
                    output_dir =OUTPUT_DIR,
                    lora_r = LORA_R,
                    lora_alpha = LORA_ALPHA,
                    lora_dropout =LORA_DROPOUT,
                    target_modules= TARGET_MODULES,
                    batch_size = BATCH_SIZE,
                    grad_accum = GRAD_ACCUM,
                    learning_rate = LR,
                    epochs  = EPOCHS,
                    max_seq_len = MAX_SEQ_LEN,
                    log_steps  = LOG_STEPS,
                    save_steps  = SAVE_STEPS,
                    hf_token  = HF_TOKEN )

Trainable parameters:
trainable params: 2,162,688 || all params: 496,195,456 || trainable%: 0.4359
Starting fine-tuning...


Step,Training Loss
10,3.833700
20,3.461400
30,3.113400
40,2.948400
50,2.789700
60,2.892200
70,2.738700
80,2.708500
90,2.684000
100,2.488400


In [58]:


# # tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, 
# #                                           use_fast=True, 
# #                                           token = HF_TOKEN)



# model = AutoModelForCausalLM.from_pretrained(
#     MODEL_ID,
#     torch_dtype=torch.float32,  
# )

# peft_config = LoraConfig(
#     r=LORA_R,
#     lora_alpha=LORA_ALPHA,
#     lora_dropout=LORA_DROPOUT,
#     bias="none",
#     task_type="CAUSAL_LM",
#     target_modules=TARGET_MODULES
# )

# model = get_peft_model(model, peft_config)

# # Explicitly enable gradients for LoRA parameters
# for name, param in model.named_parameters():
#     if 'lora' in name.lower():
#         param.requires_grad_(True)
#         # print(f"Enabled gradients for: {name}")

# print("Trainable parameters:")
# model.print_trainable_parameters()
# # Training arguments - GPU optimized
# training_args = TrainingArguments(
#     output_dir=OUTPUT_DIR,
#     per_device_train_batch_size=BATCH_SIZE,
#     per_device_eval_batch_size=2,  # Reduced for float32 memory usage
#     gradient_accumulation_steps=GRAD_ACCUM,
#     learning_rate=LR,
#     num_train_epochs=EPOCHS,
#     logging_steps=LOG_STEPS,
#     save_steps=SAVE_STEPS,
#     save_total_limit=2,
#     lr_scheduler_type="cosine",
#     warmup_ratio=0.03,
#     load_best_model_at_end=False,
#     gradient_checkpointing=False,  # Disable to avoid potential LoRA conflicts
#     report_to="none",
#     # Disable mixed precision for now to avoid LoRA compatibility issues
#     dataloader_pin_memory=True,  # Enable for GPU
#     dataloader_num_workers=0,  # Set to 0 to avoid potential issues
#     remove_unused_columns=False,
#     prediction_loss_only=True,
#     eval_strategy="no",  # Disable evaluation during training for speed
# )
# # Data collator
# data_collator = PromptAnswerCollator(tokenizer, max_seq_len=MAX_SEQ_LEN)


# # Trainer
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_ds,
#     data_collator=data_collator,
# )

# # Train the model
# print("Starting fine-tuning...")
# trainer.train()




In [42]:

# Step 3: Evaluate fine-tuned model
print("Generating fine-tuned predictions...")
id_finetuned_preds = generate_answers_with(model, tokenizer, id_val_obs, id_val_qs)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Generating fine-tuned predictions...


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info

In [43]:
od_finetuned_preds = generate_answers_with(model, tokenizer, od_val_obs, od_val_qs)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info

In [44]:
id_finetuned_metrics = eval_metrics(id_finetuned_preds, id_val_obs, rouge, bleu)
od_finetuned_metrics = eval_metrics(od_finetuned_preds, od_val_obs, rouge, bleu)


In [45]:
print("Fine-tuned metrics:", id_finetuned_metrics)


Fine-tuned metrics: {'rouge1': np.float64(0.024099290900503435), 'rouge2': np.float64(0.0009311746571898738), 'rougeL': np.float64(0.017322943360469713), 'rougeLsum': np.float64(0.017167616121636756), 'bleu': 0.016532110636621765}


In [46]:
print("Fine-tuned metrics:", od_finetuned_metrics)


Fine-tuned metrics: {'rouge1': np.float64(0.007731598379293106), 'rouge2': np.float64(0.000436246355048571), 'rougeL': np.float64(0.005793272280243998), 'rougeLsum': np.float64(0.006054111972589283), 'bleu': 2.392197299690141e-07}


In [47]:

# Save fine-tuned predictions
with open(os.path.join(OUTPUT_DIR, "id_val_predictions_finetuned.jsonl"), "w") as f:
    for obs, q, ref, pred in zip(id_val_obs, id_val_qs, id_val_refs, id_finetuned_preds):
        f.write(json.dumps({
            "Training Observations": obs,
            "Question": q,
            "Reference": ref,
            "Prediction": pred
        }, ensure_ascii=False) + "\n")

# Save model and tokenizer
model.save_pretrained(os.path.join(OUTPUT_DIR, "adapter"))
tokenizer.save_pretrained(os.path.join(OUTPUT_DIR, "adapter"))


('swiss_apertus_8b_ft_inductive_smalllm_compare/adapter/tokenizer_config.json',
 'swiss_apertus_8b_ft_inductive_smalllm_compare/adapter/special_tokens_map.json',
 'swiss_apertus_8b_ft_inductive_smalllm_compare/adapter/chat_template.jinja',
 'swiss_apertus_8b_ft_inductive_smalllm_compare/adapter/vocab.json',
 'swiss_apertus_8b_ft_inductive_smalllm_compare/adapter/merges.txt',
 'swiss_apertus_8b_ft_inductive_smalllm_compare/adapter/added_tokens.json',
 'swiss_apertus_8b_ft_inductive_smalllm_compare/adapter/tokenizer.json')

In [48]:
# Save fine-tuned predictions
with open(os.path.join(OUTPUT_DIR, "od_val_predictions_finetuned.jsonl"), "w") as f:
    for obs, q, ref, pred in zip(od_val_obs, od_val_qs, od_val_refs, od_finetuned_preds):
        f.write(json.dumps({
            "Training Observations": obs,
            "Question": q,
            "Reference": ref,
            "Prediction": pred
        }, ensure_ascii=False) + "\n")

# Save model and tokenizer
model.save_pretrained(os.path.join(OUTPUT_DIR, "adapter"))
tokenizer.save_pretrained(os.path.join(OUTPUT_DIR, "adapter"))

('swiss_apertus_8b_ft_inductive_smalllm_compare/adapter/tokenizer_config.json',
 'swiss_apertus_8b_ft_inductive_smalllm_compare/adapter/special_tokens_map.json',
 'swiss_apertus_8b_ft_inductive_smalllm_compare/adapter/chat_template.jinja',
 'swiss_apertus_8b_ft_inductive_smalllm_compare/adapter/vocab.json',
 'swiss_apertus_8b_ft_inductive_smalllm_compare/adapter/merges.txt',
 'swiss_apertus_8b_ft_inductive_smalllm_compare/adapter/added_tokens.json',
 'swiss_apertus_8b_ft_inductive_smalllm_compare/adapter/tokenizer.json')

In [49]:


# Print comparison
print_compare(id_baseline_metrics, id_finetuned_metrics)


===== OOS Metrics (Baseline vs Fine-tuned) =====
rouge1      base:  0.027   ft:  0.024   Δ: -0.003
rouge2      base:  0.002   ft:  0.001   Δ: -0.001
rougeL      base:  0.023   ft:  0.017   Δ: -0.005
rougeLsum   base:  0.023   ft:  0.017   Δ: -0.006
bleu        base:  0.021   ft:  0.017   Δ: -0.004


In [50]:


# Print comparison
print_compare(od_baseline_metrics, od_finetuned_metrics)


===== OOS Metrics (Baseline vs Fine-tuned) =====
rouge1      base:  0.008   ft:  0.008   Δ: -0.001
rouge2      base:  0.001   ft:  0.000   Δ: -0.000
rougeL      base:  0.006   ft:  0.006   Δ: -0.000
rougeLsum   base:  0.006   ft:  0.006   Δ: +0.000
bleu        base:  0.028   ft:  0.000   Δ: -0.028


In [51]:
import weightwatcher as ww
base_watcher = ww.WeightWatcher(model=baseline_model)
base_details = base_watcher.analyze()
base_summary = base_watcher.get_summary(base_details)

Running on macOS.
Running on macOS.


In [52]:
watcher = ww.WeightWatcher(model=model)
details = watcher.analyze()
summary = watcher.get_summary(details)

In [53]:
base_summary

{'log_norm': np.float64(2.557011775868817),
 'alpha': np.float64(6.332740802794254),
 'alpha_weighted': np.float64(3.8719353960280394),
 'log_alpha_norm': np.float64(4.251105918200468),
 'log_spectral_norm': np.float64(0.7483215650030496),
 'stable_rank': np.float64(84.57115935377053)}

In [54]:
summary

{'log_norm': np.float64(1.255493528534669),
 'alpha': np.float64(6.1076810384503055),
 'alpha_weighted': np.float64(0.44076106341394844),
 'log_alpha_norm': np.float64(0.7971073362406768),
 'log_spectral_norm': np.float64(-0.05658528760085752),
 'stable_rank': np.float64(44.86047495658877)}